In [1]:
from dotenv import load_dotenv
from huggingface_hub import login
import os

load_dotenv()
hftoken = os.getenv("HUGGINGFACE_TOKEN")
login(hftoken)

In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm
import time
from tqdm import tqdm

# OpenAI API key 선언
#os.environ["OPENAI_API_KEY"] = "sk-xxx..."

1. query 컬럼으로 질문 생성 및 생성된 질문에 대한 답변 생성

In [7]:

def random_wait(min_seconds=0.1, max_seconds=0.5):
    wait_time = random.uniform(min_seconds, max_seconds)
    #print(f"Waiting for {wait_time:.2f} seconds...")
    time.sleep(wait_time)

# 데이터셋 로드
ds = load_dataset('amphora/rewrite-se-quant')['train']

# 전체 데이터셋을 미리 데이터프레임으로 변환
df_ds = pd.DataFrame(ds)

# 데이터프레임 초기화
df = pd.DataFrame(columns=['query', 'question', 'response'])


batch_size = 200
total_length = len(ds['query'])
csv_file = 'output_dataset.csv'

# 반복문 안에서 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가 8300 ~ 8800까지 다시하기
for i in tqdm(range(15000, total_length, batch_size), desc="Processing Batches", unit="batch"):
    batch_end = min(i + batch_size, total_length)  # 끝 인덱스가 전체 길이를 넘지 않도록 설정

    # 질문 생성용 prompt 포맷팅
    batch_qrys = []
    for t in df_ds['query'][i:batch_end]:
        messages = [
            {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a English QF question collected from the web. Restructure it to a test-like question, in formal Korean language. Return the question only.", "role": "system"},
            {"content": t, "role": "user"}
        ]
        batch_qrys.append(messages)

    print(i, 'question creating...')

    # 질문 생성
    responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_qrys,
        # max_tokens=60
    )

    question_resps = []
    for response in responses:
        if isinstance(response, litellm.RateLimitError):
            print(f"오류 발생: {str(response)}")
            question_resps.append("Error: Rate limit exceeded or other issue")
        elif hasattr(response, 'choices') and len(response.choices) > 0:
            question_resps.append(response.choices[0].message.content)
        else:
            question_resps.append("Error: Unexpected response format")

    # 질문 생성과 답변 생성 사이에 10초 간격 추가
    print(i, '1분 대기')
    time.sleep(60)
    random_wait()

    print(i, 'response creating...')
    # 답변 생성용 prompt 포맷팅
    batch_qrys = []
    for t in question_resps:
        messages = [
            {"content": "You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.", "role": "system"},
            {"content": t, "role": "user"}
        ]
        batch_qrys.append(messages)

    # 답변 생성
    responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_qrys,
        # max_tokens=80
    )

    response_resps = []
    for resp in responses:  # 'i' 대신 'resp'로 변수명 변경
        if isinstance(resp, Exception):
            print(f"오류 발생: {str(resp)}")
            response_resps.append("Error: Rate limit exceeded or other issue")
        elif hasattr(resp, 'choices') and len(resp.choices) > 0:
            response_resps.append(resp.choices[0].message.content)
        else:
            response_resps.append("Error: Unexpected response format")


    print(i, '1분 대기')
    time.sleep(60)
    # print('대기 종료')
    random_wait()

    query_list = df_ds['query'].iloc[i:batch_end].tolist()

    # 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가
    batch_df = pd.DataFrame({
        'query': query_list,
        'question': question_resps,
        'response': response_resps
    })

    # batch_df.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8-sig')
    
    df = pd.concat([df, batch_df], ignore_index=True)

    # 다음 배치를 처리하기 전 1분 간격 대기
    # print(i, '1초 대기')
    # time.sleep(1)
    # print('대기 종료')
    random_wait()

# 최종 결과 확인
df.head()

Processing Batches:   0%|          | 0/36 [00:00<?, ?batch/s]

14800 question creating...


/home/elicer/.local/lib/python3.10/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CompletionTokensDetails` but got `dict` with value `{'audio_tokens': None, 'reasoning_tokens': 0}` - serialized value may not be as expected
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/elicer/.local/lib/python3.10/site-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CompletionTokensDetails` but got `dict` with value `{'audio_tokens': None, 'reasoning_tokens': 0}` - serialized value may not be as expected
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 1280}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


14800 1분 대기
14800 response creating...
14800 1분 대기


Processing Batches:   3%|▎         | 1/36 [02:34<1:29:58, 154.24s/batch]

15000 question creating...
15000 1분 대기
15000 response creating...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback /

Processing Batches:   3%|▎         | 1/36 [04:57<2:53:20, 297.14s/batch]


KeyboardInterrupt: 

In [8]:
df

,query,question,response
0,A utility function $U$ whose corresponding rel...,"주어진 효용 함수 \( U \)의 상대위험회피성 함수가 선형 증가 함수일 때, 다음...",주어진 미분 방정식은 다음과 같습니다:\n\n\[\n-x\frac{U''(x)}{U...
1,What is the proper way to calculate volatility...,롱-숏 포트폴리오의 변동성을 올바르게 계산하는 방법은 무엇입니까? 표준적인 방식으로...,롱-숏 포트폴리오의 변동성을 정확하게 계산하는 방법은 주로 포트폴리오에 포함된 자산...
2,Do vega and gamma always have the same sign (i...,"베가(Vega)와 감마(Gamma)는 항상 같은 부호(즉, 둘 다 양수이거나 둘 다...","베가(Vega)와 감마(Gamma)는 일반적으로 같은 부호를 가지지만, 반드시 항상..."
3,Is there an equation or rule of thumb to deter...,특정 이자율을 가진 대출에 대한 채무 불이행 확률을 결정하기 위한 공식이나 경험적 ...,채무 불이행 확률을 결정하는 공식이나 경험적 법칙은 일반적으로 여러 요소를 고려해야...
4,How can a day trader figure out what event mig...,주식 시장에서 일일 거래자(데이 트레이더)는 거래 시간 외에 발생한 갑작스러운 거래...,일일 거래자(데이 트레이더)가 거래 시간 외에 갑작스러운 거래량 급증의 원인을 파악...
...,...,...,...
195,I am looking at the Equity Option example of Q...,QuantLib의 Equity Option 예제와 관련하여 FDAmericanEng...,Finite Difference Method (FDM)을 사용하여 옵션의 전체 가치...
196,What kind of indicators may have predicted the...,2000년 또는 2008년의 금융 위기를 예측할 수 있었던 지표에는 어떤 것들이 있...,2000년과 2008년의 금융 위기를 예측할 수 있었던 몇 가지 주요 지표를 살펴보...
197,"Suppose that we have a market with a stock, mo...",주식 $\{S_t\}_{t>0}$와 무위험 자산 계좌 $\{B_t\}_{t>0}$가...,주식과 무위험 자산으로 구성된 포트폴리오에서 사용하는 약어는 투자 전략의 다른 측면...
198,At the moment the backtester has a portfolio; ...,"현재 백테스터는 하나의 포트폴리오를 가지고 있으며, 이 포트폴리오는 하나의 전략과 ...","백테스터가 각기 다른 전략을 테스트할 때, 하나의 포트폴리오에 여러 전략을 동시에 ..."


In [9]:
df.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8-sig')

In [10]:
csv_file = 'output_dataset.csv'
len(pd.read_csv(csv_file))

7066

In [22]:
df

,query,question,response
0,Every so often I hear people referring to the ...,"금융에서 ""첫 번째 원칙(first principles)""이라는 용어는 보편적으로 ...","금융에서 ""첫 번째 원칙""은 여러 논리적 또는 수학적 원리로부터 도출될 수 없는 기..."
1,I think this will be an easy question for prac...,다음의 질문에 답하십시오: 효과적인 연방기금금리는 예치기관들이 연방준비은행에 보유하...,효율적인 연방기금금리가 리포 금리보다 낮은 이유는 주로 두 가지로 해석할 수 있습니...
2,I am currently studying optimal stopping time....,최적 정지 시간에 대한 이해를 돕기 위해 다음의 질문에 답하십시오.\n\n1. 정지...,1. 정지 시간 \(\tau\)는 특정 시점에서 의사결정을 내리기 위해 기다리는 최...
3,Statement: if $c(t)$ is the price of the digit...,다음 문장을 바탕으로 문제를 작성하시오: \n\n$ c(t) $가 디지털 현금-없음...,이 문제를 증명하기 위해 Black-Scholes 모델의 옵션 가격 결정 과정을 바...
4,"In the following example, for 3rd question and...","다음 예시에서, 3번 및 4번 질문에서 왜 현재 주가에 (3개월 후 주가 - 현재 ...",3번 및 4번 질문에서 현재 주가에 (3개월 후 주가 - 현재 주가)를 더하는 이유...
...,...,...,...
2795,I am looking for intraday option price data fo...,"유럽 시장(SX5E, SMI, DAX 등)에 상장된 주식 및 지수의 하루 intra...","유럽 시장의 인트라데이 옵션 가격 데이터를 찾는 것은 다소 어려울 수 있지만, 몇 ..."
2796,"If I want to simulate a process (say, a set of...","실제 측정하에서 프로세스(예: 일련의 선도 금리)를 시뮬레이션하고자 할 때, 옵션 ...",옵션 가격이나 내재 변동성을 사용하여 프로세스를 보정하는 것은 금융 모델링에서 중요...
2797,Suppose we have a semiannual coupon bond. The ...,다음과 같은 세미연간 이자 지급 채권이 있다고 가정하겠습니다. 평가일자는 2017년...,"1. 주어진 이자 지급 주기를 사용하여 평가일과 기준일을 입력했을 때, 연간 분수를..."
2798,This is a question about modelling the returns...,채권 지수의 수익률 모형화에 관한 문제입니다. 개별 채권의 롤(Roll) 및 캐리(...,"1. 롤(Roll)는 채권 지수 수익률에 영향을 미치는 중요한 요소입니다. 이때, ..."


In [21]:
df = df[:2800]

In [20]:
df[df['response'] == 'Error: Rate limit exceeded or other issue']

,query,question,response
2821,I want to calculate the price at $t$ for such ...,다음의 지급금 형태에 대해 $t$ 시점에서 가격을 계산하고자 합니다:\n\n1. $...,Error: Rate limit exceeded or other issue
2824,I want to assume I am in a general Black Schol...,다음 질문에 답하시오.\n\n당신은 일반적인 Black-Scholes 모형을 가정하...,Error: Rate limit exceeded or other issue
2827,Where could I get historical record of listed ...,아시아 국가 및 미국의 상장 회사 제출 마감일에 대한 역사적 기록을 어디에서 찾을 ...,Error: Rate limit exceeded or other issue
2828,"In some years, some accounting values of some ...","일부 연도에 Compustat 및 CRSP에서 특정 기업의 회계 값이 누락된 경우,...",Error: Rate limit exceeded or other issue
2829,How reasonable it is to assume that the forwar...,유럽식 옵션의 콜-풋 패리티를 통해 도출된 선물 가격이 쉽게 대출할 수 있는 유럽 ...,Error: Rate limit exceeded or other issue
...,...,...,...
2895,I have two brokers who give me delta strategie...,두 명의 중개인이 제공하는 USD-COP에 대한 델타 전략이 있습니다. 한 중개인은...,Error: Rate limit exceeded or other issue
2896,we are currently looking to build another data...,"현재 우리가 만들고자 하는 데이터베이스는 SEC를 크롤링하여 수집된 것으로, Sim...",Error: Rate limit exceeded or other issue
2897,"$Let \space X(t) =\begin{cases}\n2, \qquad\tex...",주어진 과정에서 다음과 같은 $X(t)$ 함수가 주어졌습니다:\n\n\[\nX(t)...,Error: Rate limit exceeded or other issue
2898,I came across this formula for the varswap PNL...,"다음 주어진 상황을 고려하십시오. 주어진 구간 \([t_i,t_{i+1}]\)에 대...",Error: Rate limit exceeded or other issue


In [48]:
df_no_duplicates = pd.concat([df_no_duplicates, df], ignore_index=True)

df_no_duplicates

,query,question,response
0,Vanilla options are traded interbank with delt...,"바닐라 옵션이 인터뱅크에서 델타 헤지를 통해 거래될 때, 델타 헤지를 위해 일반적으...","바닐라 옵션 거래에서 델타 헤지란, 옵션 포지션의 델타를 중립화하여 가격 변동에 의..."
1,"Can the Hodges-Tompkins adjustment (S. Hodges,...","Hodges-Tompkins 조정(S. Hodges, R. Tompkins, The...",Hodges-Tompkins 조정은 중복 관측치로부터 계산된 추정량의 편향을 제거할...
2,To price a call/put option with two possible f...,"다음 질문에 답하십시오:\n\n1) 콜/풋 옵션의 가격을 결정할 때, 왜 항상 기초...",1) 콜/풋 옵션의 가격을 결정할 때 기초 자산과 무위험 채권으로 구성된 복제 포트...
3,I am trying to understand the abbreviations in...,블룸버그 유럽 주식 또는 STOXX 데이터에서 사용하는 약어에 대해 이해하려고 합니...,유럽의 다양한 증권 거래소에서 사용되는 두 글자 약어에 대한 정보를 찾고 계시군요....
4,Assume a model defined in a incomplete market....,불완전한 시장에서 정의된 모델을 가정하십시오. 이 모델에는 몇 가지 매개변수 $\t...,시장 위험 프리미엄 $\lambda$가 관찰할 수 없는 주관적인 매개변수라는 것은 ...
...,...,...,...
413,So I was trying to estimate the performance of...,전기 시장에서 정적 헤지와 동적 헤지의 성과를 추정하는 과정에서 이상한 결과를 얻었...,"첫 번째 질문에 대해 말씀드리자면, R-제곱 값이 상이하게 나타나는 이유는 회귀 분..."
414,Having to implement a replication strategy for...,유럽형 옵션의 복제 전략을 구현하는 과정에서 다음과 같은 문제가 발생하였습니다.\n...,"유럽형 옵션의 만기 시점 T에서는 옵션의 가치가 즉시 결정되며, 그에 따라 델타도 ..."
415,I am trying to bootstrap the 6m sterling swap ...,"6개월 스털링 스왑 곡선을 부트스트랩하기 위해, 주어진 예제에서 FRA(금리선도계약...",위 코드에서 FRA(금리선도계약)를 추가하기 위해서는 FRA 헬퍼를 생성할 수 있는...
416,I would like to price a n-th to default swap o...,n번째 디폴트 스왑을 5개의 자산 집합에 대해 가격을 책정하고자 합니다. R 코드로...,디폴트 스왑 가격 책정을 위해 시뮬레이션된 기본값 시리즈를 활용하는 방법에 대한 구...


In [42]:
df[df['response'] == 'Error: Rate limit exceeded or other issue']

,query,question,response
113,I was recently reading (and very much struggli...,"CBOE 스큐 지수에 대한 백서에서 설명된 내용을 바탕으로, 다음 질문에 답하십시오...",Error: Rate limit exceeded or other issue


In [30]:
df_no_duplicates = df[~df['response'].duplicated(keep=False)]

len(df_no_duplicates)

48

In [16]:
df[df.duplicated(subset='response')]

,query,question,response
3587,"Can the Hodges-Tompkins adjustment (S. Hodges,...","Hodges-Tompkins 조정(S. Hodges, R. Tompkins, The...",Error: Rate limit exceeded or other issue
3588,To price a call/put option with two possible f...,다음 질문에 답하십시오:\n\n1) 두 가지 가능한 미래 상태를 고려하여 콜/풋 옵...,Error: Rate limit exceeded or other issue
3589,I am trying to understand the abbreviations in...,유럽 주식 또는 STOXX 데이터에서 사용되는 약어에 대한 이해를 돕기 위해 다음 ...,Error: Rate limit exceeded or other issue
3590,Assume a model defined in a incomplete market....,불완전한 시장에서 정의된 모형을 가정하십시오. 이 모형은 매개변수 $\theta_1...,Error: Rate limit exceeded or other issue
3591,Examples like new research or findings but als...,"기계 학습이 금융 분야에 점차 확산되고 있는지에 대한 연구 현황을 고려할 때, 현재...",Error: Rate limit exceeded or other issue
...,...,...,...
3995,I need help in getting a more intuitive unders...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue
3996,I am looking at the process \n$$X_t = \int_0^t...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue
3997,If I made a GTC (good till cancelled) to a Sto...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue
3998,I´m trying to build a Stock ranking/Picking mo...,Error: Rate limit exceeded or other issue,Error: Rate limit exceeded or other issue


2. Qwen으로 질문 및 답변 생성 (너무 오래걸림)

In [2]:
import pandas as pd
import time
from tqdm import tqdm
from datasets import load_dataset
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def qwen_completion(model, tokenizer, inputs, max_new_tokens=60, device='cuda'):
    results = []
    for prompt in inputs:
        # Tokenize the prompt and move tensors to the appropriate device (CPU or GPU)
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():  # Ensure no unnecessary gradients are stored
            # Generate text using max_new_tokens
            outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)

        # Decode the generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results.append(generated_text)

        # Explicitly free memory
        del inputs, outputs
        torch.cuda.empty_cache()  # Free up unused memory in the GPU

    return results

# 데이터셋 로드
ds = load_dataset('amphora/rewrite-se-quant')['train']

# 전체 데이터셋을 미리 데이터프레임으로 변환
df_ds = pd.DataFrame(ds)

# 데이터프레임 초기화 ('query' 컬럼 제거)
df = pd.DataFrame(columns=['question', 'response'])

# 모델과 토크나이저를 한 번만 로드
model_name = "Qwen/Qwen2-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)

# 배치 크기 설정
batch_size = 50  # Adjust this batch size according to your GPU memory capacity

# 배치별로 질문과 답변 생성
for i in tqdm(range(0, len(df_ds), batch_size), desc="Processing Batches"):
    # 현재 배치의 데이터 선택
    batch_queries = df_ds['query'][i:i + batch_size].tolist()

    # 질문 생성 - Qwen 모델 사용 (GPU 활용)
    batch_questions = []
    for query in batch_queries:
        prompt = f"문제: {query}\n질문을 한국어로 만들어주세요."
        batch_questions.append(prompt)

    # 생성된 질문
    question_resps = qwen_completion(
        model=model,
        tokenizer=tokenizer,
        inputs=batch_questions,
        max_new_tokens=60,
        device=device
    )

    # 답변 생성용 프롬프트 생성
    batch_responses = []
    for question in question_resps:
        prompt = f"질문: {question}\n답변을 만들어주세요."
        batch_responses.append(prompt)

    # 생성된 답변
    response_resps = qwen_completion(
        model=model,
        tokenizer=tokenizer,
        inputs=batch_responses,
        max_new_tokens=80,
        device=device
    )

    # 현재 배치의 데이터프레임 생성 및 기존 데이터프레임에 추가
    batch_df = pd.DataFrame({
        'question': question_resps,
        'response': response_resps
    })
    df = pd.concat([df, batch_df], ignore_index=True)

# 최종 결과 확인
df.head()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Batches:   5%|▌         | 22/439 [2:11:37<43:17:38, 373.76s/it]

In [ ]:
print(f"중복된 행의 개수: {df.duplicated().sum()}")

In [ ]:
# 질문 생성 결과 길이 맞추기
if len(question_resps) < len(ds["query"]):
    question_resps.extend(["Error: Missing question"] * (len(ds["query"]) - len(question_resps)))

# 답변 생성 결과 길이 맞추기
if len(response_resps) < len(ds["query"]):
    response_resps.extend(["Error: Missing response"] * (len(ds["query"]) - len(response_resps)))


In [49]:
# Excel 파일 저장
# df.to_excel("output_path/result.xlsx")

# HuggingFace Hub 업로드 - token에 개인 HuggingFace 토큰을 입력해주시면 됩니다.
result_df = Dataset.from_pandas(df_no_duplicates)
result_df.push_to_hub("llmflow/krx_dataset3", token=hftoken)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/llmflow/krx_dataset3/commit/f58317afe34a8c7d33c2415ec5c9f77831ec5a63', commit_message='Upload dataset', commit_description='', oid='f58317afe34a8c7d33c2415ec5c9f77831ec5a63', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/llmflow/krx_dataset3', endpoint='https://huggingface.co', repo_type='dataset', repo_id='llmflow/krx_dataset3'), pr_revision=None, pr_num=None)